In [1]:
import sys
import numpy as np
import pandas as pd
import matplotlib as mpl
import sklearn
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [2]:
years = ['2022']
months = ['01','02','03','04','05','06','07','08','09','10','11','12']

arr = ['202302','202301']

for year in years:
    for month in months:
        arr.append(year+month)

dfs = []

for i in arr:
    df = pd.read_csv(f'house{i}.csv',encoding= 'cp949')
    dfs.append(df)

In [3]:
dfs

[                시군구   전용면적(㎡)    계약년월 거래금액(만원)   층    건축년도
 0       강원도 강릉시 견소동   84.9450  202302   25,500   4  1997.0
 1       강원도 강릉시 견소동   84.9450  202302   16,000   1  1997.0
 2       강원도 강릉시 견소동   84.9450  202302   23,000   1  1997.0
 3       강원도 강릉시 견소동   84.9900  202302   29,000   6  2005.0
 4        강원도 강릉시 교동  164.0098  202302   59,500  12  2009.0
 ...             ...       ...     ...      ...  ..     ...
 32600  충청북도 충주시 호암동   84.9289  202302   23,000   1  2007.0
 32601  충청북도 충주시 호암동   84.9289  202302   23,000   7  2007.0
 32602  충청북도 충주시 호암동   84.6800  202302   20,800   3  2002.0
 32603  충청북도 충주시 호암동   84.9500  202302   40,000  16  2019.0
 32604  충청북도 충주시 호암동   99.8000  202302   49,000  11  2019.0
 
 [32605 rows x 6 columns],
                 시군구  전용면적(㎡)    계약년월 거래금액(만원)   층    건축년도
 0       강원도 강릉시 견소동   84.930  202301   25,000   6  1997.0
 1       강원도 강릉시 견소동   84.990  202301   29,800   8  2005.0
 2       강원도 강릉시 견소동   84.990  202301   28,500   5  2005.0
 3        강원도 강

In [4]:
total_df = pd.DataFrame()
for df in dfs:
    total_df = pd.concat([total_df,df])

In [5]:
total_df

,시군구,전용면적(㎡),계약년월,거래금액(만원),층,건축년도
0,강원도 강릉시 견소동,84.9450,202302,"25,500",4,1997.0
1,강원도 강릉시 견소동,84.9450,202302,"16,000",1,1997.0
2,강원도 강릉시 견소동,84.9450,202302,"23,000",1,1997.0
3,강원도 강릉시 견소동,84.9900,202302,"29,000",6,2005.0
4,강원도 강릉시 교동,164.0098,202302,"59,500",12,2009.0
...,...,...,...,...,...,...
15425,충청북도 충주시 호암동,84.8030,202212,"39,300",23,2019.0
15426,충청북도 충주시 호암동,84.9600,202212,"17,500",2,1994.0
15427,충청북도 충주시 호암동,84.9289,202212,"23,500",4,2007.0
15428,충청북도 충주시 호암동,84.9400,202212,"40,000",11,2019.0


In [6]:
total_df['큰지역'] = total_df['시군구'].apply(lambda x:x.split()[0])
total_df['세부지역'] = total_df['시군구'].apply(lambda x:x.split()[1])
total_df.drop(columns='시군구', inplace=True)

In [7]:
total_df = total_df[['거래금액(만원)','전용면적(㎡)','계약년월','층','건축년도','큰지역','세부지역'] ]

In [8]:
total_df.info() # 건축년도에 36개 null 존재

<class 'pandas.core.frame.DataFrame'>
Int64Index: 325133 entries, 0 to 15429
Data columns (total 7 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   거래금액(만원)  325133 non-null  object 
 1   전용면적(㎡)   325133 non-null  float64
 2   계약년월      325133 non-null  int64  
 3   층         325133 non-null  int64  
 4   건축년도      325097 non-null  float64
 5   큰지역       325133 non-null  object 
 6   세부지역      325133 non-null  object 
dtypes: float64(2), int64(2), object(3)
memory usage: 19.8+ MB


In [9]:
total_df = total_df.dropna(axis=0) #null 제거

In [10]:
total_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 325097 entries, 0 to 15429
Data columns (total 7 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   거래금액(만원)  325097 non-null  object 
 1   전용면적(㎡)   325097 non-null  float64
 2   계약년월      325097 non-null  int64  
 3   층         325097 non-null  int64  
 4   건축년도      325097 non-null  float64
 5   큰지역       325097 non-null  object 
 6   세부지역      325097 non-null  object 
dtypes: float64(2), int64(2), object(3)
memory usage: 19.8+ MB


In [11]:
total_df['거래금액(만원)'] # ,때문에 type이 object로 나옴

0        25,500
1        16,000
2        23,000
3        29,000
4        59,500
          ...  
15425    39,300
15426    17,500
15427    23,500
15428    40,000
15429    55,000
Name: 거래금액(만원), Length: 325097, dtype: object

In [50]:
total_df['거래금액(만원)'] = total_df['거래금액(만원)'].apply(lambda x: x.split(',')[0] + x.split(',')[1] if len(x.split(',')) >= 2  else x.split(',')[0]) # ,제거 object -> int
total_df['거래금액(만원)'] = total_df['거래금액(만원)'].astype('int')
total_df.info()

AttributeError: 'int' object has no attribute 'split'

In [ ]:
total_df['큰지역'].value_counts()

In [81]:
local_GS = (total_df.큰지역=='경기도') | (total_df.큰지역=='서울특별시')
GS_df = total_df[local_GS]
GS_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 79600 entries, 1068 to 10433
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   거래금액(만원)  79600 non-null  int64  
 1   전용면적(㎡)   79600 non-null  float64
 2   계약년월      79600 non-null  int64  
 3   층         79600 non-null  int64  
 4   건축년도      79600 non-null  float64
 5   큰지역       79600 non-null  object 
 6   세부지역      79600 non-null  object 
dtypes: float64(2), int64(3), object(2)
memory usage: 4.9+ MB


In [82]:
GS_df[GS_df.큰지역 == '경기도']

,거래금액(만원),전용면적(㎡),계약년월,층,건축년도,큰지역,세부지역
1068,6000,59.4900,202302,6,1992.0,경기도,가평군
1070,17850,84.8420,202302,2,2007.0,경기도,가평군
1071,7000,75.3600,202302,1,1991.0,경기도,가평군
1072,32700,127.9437,202302,2,2008.0,경기도,가평군
1073,20700,59.8700,202302,13,2018.0,경기도,가평군
...,...,...,...,...,...,...,...
3952,34500,84.8578,202212,6,2008.0,경기도,화성시
3953,33000,84.8578,202212,6,2008.0,경기도,화성시
3954,39500,84.9730,202212,12,2008.0,경기도,화성시
3955,43000,84.9870,202212,20,2008.0,경기도,화성시


In [83]:
GS_df.corr(method = 'pearson')

/var/folders/0r/1nt20vf52jx576xwtk9wl2lw0000gn/T/ipykernel_15025/2711072175.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  GS_df.corr(method = 'pearson')


,거래금액(만원),전용면적(㎡),계약년월,층,건축년도
거래금액(만원),1.000000,0.523338,0.030970,0.181920,0.040460
전용면적(㎡),0.523338,1.000000,0.066849,0.141407,0.024048
계약년월,0.030970,0.066849,1.000000,0.083440,0.108917
층,0.181920,0.141407,0.083440,1.000000,0.287387
건축년도,0.040460,0.024048,0.108917,0.287387,1.000000


In [84]:
GS_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 79600 entries, 1068 to 10433
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   거래금액(만원)  79600 non-null  int64  
 1   전용면적(㎡)   79600 non-null  float64
 2   계약년월      79600 non-null  int64  
 3   층         79600 non-null  int64  
 4   건축년도      79600 non-null  float64
 5   큰지역       79600 non-null  object 
 6   세부지역      79600 non-null  object 
dtypes: float64(2), int64(3), object(2)
memory usage: 4.9+ MB


In [88]:
mapping_dict = {'경기도': 0, '서울특별시': 1}
GS_df['큰지역'] = GS_df['큰지역'].map(mapping_dict)

mapping_dict = {'가평군': 0,
 '고양덕양구': 1,
 '고양일산동구': 2,
 '고양일산서구': 3,
 '과천시': 4,
 '광명시': 5,
 '광주시': 6,
 '구리시': 7,
 '군포시': 8,
 '김포시': 9,
 '남양주시': 10,
 '동두천시': 11,
 '부천시': 12,
 '성남분당구': 13,
 '성남수정구': 14,
 '성남중원구': 15,
 '수원권선구': 16,
 '수원영통구': 17,
 '수원장안구': 18,
 '수원팔달구': 19,
 '시흥시': 20,
 '안산단원구': 21,
 '안산상록구': 22,
 '안성시': 23,
 '안양동안구': 24,
 '안양만안구': 25,
 '양주시': 26,
 '양평군': 27,
 '여주시': 28,
 '연천군': 29,
 '오산시': 30,
 '용인기흥구': 31,
 '용인수지구': 32,
 '용인처인구': 33,
 '의왕시': 34,
 '의정부시': 35,
 '이천시': 36,
 '파주시': 37,
 '평택시': 38,
 '포천시': 39,
 '하남시': 40,
 '화성시': 41,
 '강남구': 42,
 '강동구': 43,
 '강북구': 44,
 '강서구': 45,
 '관악구': 46,
 '광진구': 47,
 '구로구': 48,
 '금천구': 49,
 '노원구': 50,
 '도봉구': 51,
 '동대문구': 52,
 '동작구': 53,
 '마포구': 54,
 '서대문구': 55,
 '서초구': 56,
 '성동구': 57,
 '성북구': 58,
 '송파구': 59,
 '양천구': 60,
 '영등포구': 61,
 '용산구': 62,
 '은평구': 63,
 '종로구': 64,
 '중구': 65,
 '중랑구': 66}
GS_df['세부지역'] = GS_df['세부지역'].map(mapping_dict)

/var/folders/0r/1nt20vf52jx576xwtk9wl2lw0000gn/T/ipykernel_15025/631463283.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  GS_df['큰지역'] = GS_df['큰지역'].map(mapping_dict)
/var/folders/0r/1nt20vf52jx576xwtk9wl2lw0000gn/T/ipykernel_15025/631463283.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  GS_df['세부지역'] = GS_df['세부지역'].map(mapping_dict)


In [90]:
GS_df['세부지역']

1068      0
1070      0
1071      0
1072      0
1073      0
         ..
10429    66
10430    66
10431    66
10432    66
10433    66
Name: 세부지역, Length: 79600, dtype: int64

In [92]:
GS_df.to_csv("/Users/kangminje/Documents/houseprice/temp.csv", mode='w', encoding="utf-8-sig")

In [40]:
GS_ohe = pd.get_dummies(GS_df[['큰지역','세부지역']])
GS_ohe

,큰지역_경기도,큰지역_서울특별시,세부지역_가평군,세부지역_강남구,세부지역_강동구,세부지역_강북구,세부지역_강서구,세부지역_고양덕양구,세부지역_고양일산동구,세부지역_고양일산서구,...,세부지역_의정부시,세부지역_이천시,세부지역_종로구,세부지역_중구,세부지역_중랑구,세부지역_파주시,세부지역_평택시,세부지역_포천시,세부지역_하남시,세부지역_화성시
1068,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1070,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1071,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1072,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1073,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10429,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
10430,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
10431,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
10432,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [19]:
GS_with_ohe = pd.concat([GS_df, GS_ohe], axis = 1) #one hot encoding
GS_with_ohe.drop(columns=['큰지역','세부지역'], inplace=True)
GS_with_ohe

,거래금액(만원),전용면적(㎡),계약년월,층,건축년도,큰지역_경기도,큰지역_서울특별시,세부지역_가평군,세부지역_강남구,세부지역_강동구,...,세부지역_의정부시,세부지역_이천시,세부지역_종로구,세부지역_중구,세부지역_중랑구,세부지역_파주시,세부지역_평택시,세부지역_포천시,세부지역_하남시,세부지역_화성시
1068,6000,59.4900,202302,6,1992.0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1070,17850,84.8420,202302,2,2007.0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1071,7000,75.3600,202302,1,1991.0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1072,32700,127.9437,202302,2,2008.0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1073,20700,59.8700,202302,13,2018.0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10429,19500,22.7000,202212,12,2019.0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
10430,19500,22.7000,202212,12,2019.0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
10431,15300,15.3100,202212,17,2018.0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
10432,13900,14.8550,202212,7,2014.0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [20]:
GS_with_ohe.columns

Index(['거래금액(만원)', '전용면적(㎡)', '계약년월', '층', '건축년도', '큰지역_경기도', '큰지역_서울특별시',
       '세부지역_가평군', '세부지역_강남구', '세부지역_강동구', '세부지역_강북구', '세부지역_강서구',
       '세부지역_고양덕양구', '세부지역_고양일산동구', '세부지역_고양일산서구', '세부지역_과천시', '세부지역_관악구',
       '세부지역_광명시', '세부지역_광주시', '세부지역_광진구', '세부지역_구로구', '세부지역_구리시', '세부지역_군포시',
       '세부지역_금천구', '세부지역_김포시', '세부지역_남양주시', '세부지역_노원구', '세부지역_도봉구',
       '세부지역_동대문구', '세부지역_동두천시', '세부지역_동작구', '세부지역_마포구', '세부지역_부천시',
       '세부지역_서대문구', '세부지역_서초구', '세부지역_성남분당구', '세부지역_성남수정구', '세부지역_성남중원구',
       '세부지역_성동구', '세부지역_성북구', '세부지역_송파구', '세부지역_수원권선구', '세부지역_수원영통구',
       '세부지역_수원장안구', '세부지역_수원팔달구', '세부지역_시흥시', '세부지역_안산단원구', '세부지역_안산상록구',
       '세부지역_안성시', '세부지역_안양동안구', '세부지역_안양만안구', '세부지역_양주시', '세부지역_양천구',
       '세부지역_양평군', '세부지역_여주시', '세부지역_연천군', '세부지역_영등포구', '세부지역_오산시', '세부지역_용산구',
       '세부지역_용인기흥구', '세부지역_용인수지구', '세부지역_용인처인구', '세부지역_은평구', '세부지역_의왕시',
       '세부지역_의정부시', '세부지역_이천시', '세부지역_종로구', '세부지역_중구', '세부지역_중랑구', '세부지역_파주시',
       '세부지역_평택시', '세부지역_포천시', '세부지역_하남시', '

In [20]:
data = GS_with_ohe.iloc[:,1:]
target = GS_with_ohe.iloc[:,:1]
x_train, x_valid, y_train, y_valid = train_test_split(data, target, test_size=0.2, shuffle=True, random_state=34)
print(len(x_train), len(x_valid))
print(len(y_train), len(y_valid))

63680 15920
63680 15920


In [21]:
GS_with_ohe.to_csv("/Users/kangminje/Documents/houseprice/GSdata.csv", mode='w', encoding="utf-8-sig")

In [23]:
from sklearn.model_selection import GridSearchCV
import xgboost as xgb

param = {
    'learning_rate':[0.05,0.07,0.1],
#     'max_depth':[5,6,7],
#     'n_estimators':range(550,700,50),
#     'colsample_bytree':[0.5,0.7,1],
#     'min_child_weight':[1,2,3],
#     'subsample':[0.6,0.7,0.8]
}
model = xgb.XGBRegressor(tree_method='gpu_hist')
grid_search = GridSearchCV(estimator=model, param_grid=param, cv=5, 
                           scoring='neg_mean_squared_error',
                           n_jobs=-1)

grid_search.fit(x_train, y_train)
print(grid_search.best_params_)
print(grid_search.best_estimator_)

ValueError: 
All the 15 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/kangminje/opt/anaconda3/envs/AI/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/kangminje/opt/anaconda3/envs/AI/lib/python3.10/site-packages/xgboost/core.py", line 620, in inner_f
    return func(**kwargs)
  File "/Users/kangminje/opt/anaconda3/envs/AI/lib/python3.10/site-packages/xgboost/sklearn.py", line 1025, in fit
    self._Booster = train(
  File "/Users/kangminje/opt/anaconda3/envs/AI/lib/python3.10/site-packages/xgboost/core.py", line 620, in inner_f
    return func(**kwargs)
  File "/Users/kangminje/opt/

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
pred_train = grid_search.predict(x_train)
pred_val = grid_search.predict(x_valid)

print('train mae score: ', mean_absolute_error(y_train, pred_train))
print('val mae score:', mean_absolute_error(y_valid, pred_val))